In [0]:
import pyspark.sql.functions as sf

In [0]:
df_shop = spark.read.table("runescape.02_silver.shop_prices_enriched")

In [0]:
# Read data from 'runescape.01_bronze.1h_prices'
df_hourly = spark.read.table("runescape.01_bronze.1h_prices")
df_hourly.count()

In [0]:
df = df_shop.join(df_hourly, "id")

In [0]:
df = df.filter("store != 'gabooty'")
df = df.withColumn("profit_sell_to_store_high", sf.col("buy_price") - sf.col("avg1HourHigh"))
df = df.withColumn("profit_sell_to_store_low", sf.col("buy_price") - sf.col("avg1Hourlow"))

In [0]:
df.display()

In [0]:
# profit for selling Amulet of Strength to Davon over time
df_aos = df.filter("id == 1725")
df_aos.display()

Databricks visualization. Run in Databricks to view.

In [0]:
# profit for selling to Davon
df_davon = df.filter("store == 'davon'")
df_davon = df_davon.filter("id <> 1718") # holy symbols are never profitable
df_davon.display()

Databricks visualization. Run in Databricks to view.